### 开始

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os

### Model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import keras_tuner
from tensorflow.keras import backend as K

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_visible_devices(devices=physical_devices[0], device_type='GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
keras.backend.set_image_data_format('channels_last')

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
#自定义matries 计算r2
def R2(y_true, y_pred):
    sst = K.sum(K.square(y_true - K.mean(y_true)))
    ssr = K.sum(K.square(y_pred - y_true))
    R2 = 1 - ssr/sst
    return R2

#### 1

In [ ]:
# Create the keras tuner model.
class MyHyperModel(keras_tuner.HyperModel):
    
    def build(self, hp):

        #drop = hp.Boolean('dropout')
        unit = hp.Choice('units',values = [100,200,300,400,500],default = 300)
        activate = hp.Choice('activate',values = ["relu", "tanh"],default = 'relu')
        model = tf.keras.Sequential()
        model.add(layers.Flatten())
        for i in range(hp.Int("num_layers", 4,6)):
            model.add(layers.Dense(units=unit,activation=activate))
            if hp.Boolean("dropout"):
                model.add(layers.Dropout(rate=0.2))
        
        model.add(layers.Dense(1,activation=hp.Choice('activate_dense_last',values = ['linear','sigmoid'])))
        learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),loss='mse',metrics=[R2,'mae'])
        return model




#### 2

In [ ]:
# Create the keras tuner model.
class MyHyperModel(keras_tuner.HyperModel):
    
    def build(self, hp):

        model = keras.Sequential()
        model.add(layers.Flatten())
        # Tune the number of layers.
        for i in range(hp.Int("num_layers",4,5)):
            model.add(
                layers.Dense(units=hp.Choice(f"units_{i}", values = [64,128,256,512]),
                             activation=hp.Choice("activation", values = ["relu", "tanh"])))
            if hp.Boolean("dropout"):
                model.add(layers.Dropout(rate=0.2))
        
        model.add(layers.Dense(1,activation=hp.Choice('activate_dense_last',values = ['linear','sigmoid'])))
        learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
        model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),loss='mse',metrics=[R2,'mae'])
        return model

In [ ]:
training_mode = 'parameter' # parameter
max_trials = 50
batch_size = 4096*4

### 2001

In [ ]:
from sklearn.preprocessing import StandardScaler
year = 2001
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

In [9]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=max_trials,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 50 Complete [00h 01m 13s]
val_R2: 0.7168009281158447

Best val_R2 So Far: 0.8153210878372192
Total elapsed time: 00h 59m 38s
INFO:tensorflow:Oracle triggered exit


In [10]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00322, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2001_best.hdf5
27/27 - 2s - loss: 0.0120 - R2: -5.7203e-01 - mae: 0.0786 - val_loss: 0.0032 - val_R2: 0.5672 - val_mae: 0.0451 - lr: 8.1201e-04 - 2s/epoch - 60ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00322 to 0.00238, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2001_best.hdf5
27/27 - 1s - loss: 0.0026 - R2: 0.6443 - mae: 0.0404 - val_loss: 0.0024 - val_R2: 0.6808 - val_mae: 0.0382 - lr: 8.1201e-04 - 991ms/epoch - 37ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00238 to 0.00210, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2001_best.hdf5
27/27 - 1s - loss: 0.0022 - R2: 0.7004 - mae: 0.0367 - val_loss: 0.0021 - val_R2: 0.7175 - val_mae: 0.0356 - lr: 8.1201e-04 - 888ms/epoch - 33ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00210 to 0.00197, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_200

In [11]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2001
{'units': 400, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0008120090241872257}
1712/1712 [==============================] - 19s 11ms/step - loss: 0.0014 - R2: 0.8036 - mae: 0.0277


[0.001363406190648675, 0.8036243319511414, 0.0276893712580204]

### 2002

In [12]:
from sklearn.preprocessing import StandardScaler
year = 2002
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (505950, 18, 1)
X_validate shape: (151785, 18, 1)


In [13]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 09s]
val_R2: 0.6332182884216309

Best val_R2 So Far: 0.8410646319389343
Total elapsed time: 01h 14m 40s
INFO:tensorflow:Oracle triggered exit


In [14]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00283, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2002_best.hdf5
31/31 - 2s - loss: 0.0091 - R2: -3.0202e-01 - mae: 0.0676 - val_loss: 0.0028 - val_R2: 0.5946 - val_mae: 0.0420 - lr: 9.1423e-04 - 2s/epoch - 80ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00283 to 0.00212, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2002_best.hdf5
31/31 - 2s - loss: 0.0024 - R2: 0.6551 - mae: 0.0380 - val_loss: 0.0021 - val_R2: 0.6966 - val_mae: 0.0356 - lr: 9.1423e-04 - 2s/epoch - 56ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00212 to 0.00187, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2002_best.hdf5
31/31 - 2s - loss: 0.0020 - R2: 0.7142 - mae: 0.0342 - val_loss: 0.0019 - val_R2: 0.7330 - val_mae: 0.0331 - lr: 9.1423e-04 - 2s/epoch - 49ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00187 to 0.00171, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2002_best

In [15]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2002
{'units': 500, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0009142311834577992}
2033/2033 [==============================] - 30s 15ms/step - loss: 0.0011 - R2: 0.8295 - mae: 0.0245


[0.0011206106282770634, 0.8295137286186218, 0.02447638474404812]

### 2003

In [16]:
from sklearn.preprocessing import StandardScaler
year = 2003
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (507932, 18, 1)
X_validate shape: (152380, 18, 1)


In [17]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 01m 56s]
val_R2: 0.8167570233345032

Best val_R2 So Far: 0.8525863885879517
Total elapsed time: 01h 11m 46s
INFO:tensorflow:Oracle triggered exit


In [18]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00257, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2003_best.hdf5
32/32 - 2s - loss: 0.0126 - R2: -7.7506e-01 - mae: 0.0735 - val_loss: 0.0026 - val_R2: 0.6291 - val_mae: 0.0400 - lr: 0.0011 - 2s/epoch - 77ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00257 to 0.00217, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2003_best.hdf5
32/32 - 2s - loss: 0.0021 - R2: 0.6880 - mae: 0.0360 - val_loss: 0.0022 - val_R2: 0.6854 - val_mae: 0.0366 - lr: 0.0011 - 2s/epoch - 64ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00217 to 0.00190, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2003_best.hdf5
32/32 - 2s - loss: 0.0018 - R2: 0.7397 - mae: 0.0327 - val_loss: 0.0019 - val_R2: 0.7248 - val_mae: 0.0337 - lr: 0.0011 - 2s/epoch - 60ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00190 to 0.00178, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2003_best.hdf5
32/32 

In [19]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2003
{'units': 500, 'activate': 'relu', 'num_layers': 6, 'dropout': False, 'activate_dense_last': 'linear', 'lr': 0.0011025533633580264}
2041/2041 [==============================] - 33s 16ms/step - loss: 0.0010 - R2: 0.8431 - mae: 0.0237


[0.0010343308094888926, 0.8430824279785156, 0.023717578500509262]

### 2004

In [20]:
from sklearn.preprocessing import StandardScaler
year = 2004
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (500833, 18, 1)
X_validate shape: (150250, 18, 1)


In [21]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 00m 38s]
val_R2: -6.6713104248046875

Best val_R2 So Far: 0.858809769153595
Total elapsed time: 01h 10m 57s
INFO:tensorflow:Oracle triggered exit


In [22]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00271, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2004_best.hdf5
31/31 - 2s - loss: 0.0082 - R2: -1.6869e-01 - mae: 0.0649 - val_loss: 0.0027 - val_R2: 0.6067 - val_mae: 0.0410 - lr: 8.8835e-04 - 2s/epoch - 65ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00271 to 0.00193, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2004_best.hdf5
31/31 - 2s - loss: 0.0022 - R2: 0.6849 - mae: 0.0362 - val_loss: 0.0019 - val_R2: 0.7209 - val_mae: 0.0339 - lr: 8.8835e-04 - 2s/epoch - 56ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00193 to 0.00171, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2004_best.hdf5
31/31 - 1s - loss: 0.0018 - R2: 0.7417 - mae: 0.0325 - val_loss: 0.0017 - val_R2: 0.7543 - val_mae: 0.0315 - lr: 8.8835e-04 - 1s/epoch - 48ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00171 to 0.00156, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2004_best

In [23]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2004
{'units': 500, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0008883465055850036}
2013/2013 [==============================] - 31s 15ms/step - loss: 9.3563e-04 - R2: 0.8548 - mae: 0.0226


[0.0009356304653920233, 0.8548129796981812, 0.02255786396563053]

### 2005

In [24]:
from sklearn.preprocessing import StandardScaler
year = 2005
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (505351, 18, 1)
X_validate shape: (151606, 18, 1)


In [25]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 00s]
val_R2: 0.6895014047622681

Best val_R2 So Far: 0.8492182493209839
Total elapsed time: 01h 15m 25s
INFO:tensorflow:Oracle triggered exit


In [26]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00369, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2005_best.hdf5
31/31 - 2s - loss: 0.0411 - R2: -4.8275e+00 - mae: 0.1092 - val_loss: 0.0037 - val_R2: 0.4758 - val_mae: 0.0483 - lr: 0.0014 - 2s/epoch - 74ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00369 to 0.00238, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2005_best.hdf5
31/31 - 1s - loss: 0.0030 - R2: 0.5783 - mae: 0.0428 - val_loss: 0.0024 - val_R2: 0.6611 - val_mae: 0.0382 - lr: 0.0014 - 1s/epoch - 46ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00238 to 0.00186, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2005_best.hdf5
31/31 - 2s - loss: 0.0021 - R2: 0.7011 - mae: 0.0356 - val_loss: 0.0019 - val_R2: 0.7347 - val_mae: 0.0334 - lr: 0.0014 - 2s/epoch - 51ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00186 to 0.00166, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2005_best.hdf5
31/31 

In [27]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2005
{'units': 500, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'linear', 'lr': 0.0013968946323924412}
2031/2031 [==============================] - 35s 17ms/step - loss: 0.0011 - R2: 0.8391 - mae: 0.0242


[0.001068287412635982, 0.839073896408081, 0.02424797974526882]

### 2006

In [28]:
from sklearn.preprocessing import StandardScaler
year = 2006
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (513265, 18, 1)
X_validate shape: (153980, 18, 1)


In [29]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 02s]
val_R2: 0.835409939289093

Best val_R2 So Far: 0.8532880544662476
Total elapsed time: 01h 28m 12s
INFO:tensorflow:Oracle triggered exit


In [30]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00237, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2006_best.hdf5
32/32 - 2s - loss: 0.0073 - R2: -6.9215e-02 - mae: 0.0586 - val_loss: 0.0024 - val_R2: 0.6477 - val_mae: 0.0377 - lr: 0.0016 - 2s/epoch - 74ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00237 to 0.00185, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2006_best.hdf5
32/32 - 1s - loss: 0.0020 - R2: 0.6959 - mae: 0.0351 - val_loss: 0.0019 - val_R2: 0.7244 - val_mae: 0.0333 - lr: 0.0016 - 1s/epoch - 43ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00185 to 0.00165, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2006_best.hdf5
32/32 - 1s - loss: 0.0018 - R2: 0.7392 - mae: 0.0321 - val_loss: 0.0017 - val_R2: 0.7542 - val_mae: 0.0311 - lr: 0.0016 - 1s/epoch - 45ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00165 to 0.00154, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2006_best.hdf5
32/32 

In [31]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2006
{'units': 500, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0016387008781448987}
2063/2063 [==============================] - 42s 20ms/step - loss: 9.9795e-04 - R2: 0.8415 - mae: 0.0233


[0.000997947296127677, 0.8414646983146667, 0.023304855450987816]

### 2007

In [32]:
from sklearn.preprocessing import StandardScaler
year = 2007
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (618216, 18, 1)
X_validate shape: (185465, 18, 1)


In [33]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 13s]
val_R2: 0.6829116344451904

Best val_R2 So Far: 0.8591227531433105
Total elapsed time: 01h 34m 59s
INFO:tensorflow:Oracle triggered exit


In [34]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00255, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2007_best.hdf5
38/38 - 2s - loss: 0.0059 - R2: 0.2262 - mae: 0.0562 - val_loss: 0.0025 - val_R2: 0.6646 - val_mae: 0.0392 - lr: 4.6633e-04 - 2s/epoch - 61ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00255 to 0.00191, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2007_best.hdf5
38/38 - 1s - loss: 0.0022 - R2: 0.7181 - mae: 0.0360 - val_loss: 0.0019 - val_R2: 0.7490 - val_mae: 0.0338 - lr: 4.6633e-04 - 1s/epoch - 38ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00191 to 0.00170, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2007_best.hdf5
38/38 - 2s - loss: 0.0018 - R2: 0.7635 - mae: 0.0327 - val_loss: 0.0017 - val_R2: 0.7761 - val_mae: 0.0317 - lr: 4.6633e-04 - 2s/epoch - 40ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00170 to 0.00158, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2007_best.hdf5

In [35]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2007
{'units': 300, 'activate': 'relu', 'num_layers': 6, 'dropout': False, 'activate_dense_last': 'linear', 'lr': 0.00046633375021358785}
2484/2484 [==============================] - 35s 14ms/step - loss: 0.0011 - R2: 0.8521 - mae: 0.0242


[0.001057443325407803, 0.8521334528923035, 0.024196363985538483]

### 2008

In [36]:
from sklearn.preprocessing import StandardScaler
year = 2008
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (625440, 18, 1)
X_validate shape: (187632, 18, 1)


In [37]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 18s]
val_R2: 0.8153479695320129

Best val_R2 So Far: 0.8806365132331848
Total elapsed time: 01h 39m 23s
INFO:tensorflow:Oracle triggered exit


In [38]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00214, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2008_best.hdf5
39/39 - 2s - loss: 0.0070 - R2: 0.1156 - mae: 0.0584 - val_loss: 0.0021 - val_R2: 0.7255 - val_mae: 0.0361 - lr: 0.0036 - 2s/epoch - 60ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00214 to 0.00167, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2008_best.hdf5
39/39 - 1s - loss: 0.0019 - R2: 0.7608 - mae: 0.0335 - val_loss: 0.0017 - val_R2: 0.7861 - val_mae: 0.0314 - lr: 0.0036 - 1s/epoch - 36ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00167 to 0.00149, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2008_best.hdf5
39/39 - 2s - loss: 0.0016 - R2: 0.7995 - mae: 0.0303 - val_loss: 0.0015 - val_R2: 0.8086 - val_mae: 0.0295 - lr: 0.0036 - 2s/epoch - 42ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00149 to 0.00141, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2008_best.hdf5
39/39 - 1s 

In [39]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2008
{'units': 200, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0035516699244168507}
2513/2513 [==============================] - 34s 13ms/step - loss: 9.4651e-04 - R2: 0.8699 - mae: 0.0229


[0.0009465068578720093, 0.8698952198028564, 0.022853553295135498]

### 2009

In [40]:
from sklearn.preprocessing import StandardScaler
year = 2009
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (613756, 18, 1)
X_validate shape: (184127, 18, 1)


In [41]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 01m 54s]
val_R2: 0.7753896117210388

Best val_R2 So Far: 0.8633864521980286
Total elapsed time: 01h 33m 20s
INFO:tensorflow:Oracle triggered exit


In [42]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00210, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2009_best.hdf5
38/38 - 2s - loss: 0.0061 - R2: 0.1885 - mae: 0.0548 - val_loss: 0.0021 - val_R2: 0.7186 - val_mae: 0.0354 - lr: 8.3681e-04 - 2s/epoch - 47ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00210 to 0.00165, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2009_best.hdf5
38/38 - 1s - loss: 0.0018 - R2: 0.7540 - mae: 0.0328 - val_loss: 0.0017 - val_R2: 0.7779 - val_mae: 0.0310 - lr: 8.3681e-04 - 1s/epoch - 34ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00165 to 0.00150, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2009_best.hdf5
38/38 - 1s - loss: 0.0016 - R2: 0.7880 - mae: 0.0303 - val_loss: 0.0015 - val_R2: 0.7983 - val_mae: 0.0296 - lr: 8.3681e-04 - 1s/epoch - 32ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00150 to 0.00144, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2009_best.hdf5

In [43]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2009
{'units': 300, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'linear', 'lr': 0.0008368139422213535}
2466/2466 [==============================] - 45s 18ms/step - loss: 9.7040e-04 - R2: 0.8615 - mae: 0.0230


[0.0009704018011689186, 0.8614882230758667, 0.023001007735729218]

### 2010

In [44]:
from sklearn.preprocessing import StandardScaler
year = 2010
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (650862, 18, 1)
X_validate shape: (195259, 18, 1)


In [45]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 23s]
val_R2: 0.861609160900116

Best val_R2 So Far: 0.8832997679710388
Total elapsed time: 01h 31m 36s
INFO:tensorflow:Oracle triggered exit


In [46]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00210, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2010_best.hdf5
40/40 - 2s - loss: 0.0062 - R2: 0.2154 - mae: 0.0536 - val_loss: 0.0021 - val_R2: 0.7307 - val_mae: 0.0352 - lr: 0.0022 - 2s/epoch - 49ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00210 to 0.00169, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2010_best.hdf5
40/40 - 1s - loss: 0.0019 - R2: 0.7612 - mae: 0.0331 - val_loss: 0.0017 - val_R2: 0.7827 - val_mae: 0.0314 - lr: 0.0022 - 1s/epoch - 33ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00169 to 0.00157, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2010_best.hdf5
40/40 - 2s - loss: 0.0016 - R2: 0.7931 - mae: 0.0305 - val_loss: 0.0016 - val_R2: 0.7989 - val_mae: 0.0298 - lr: 0.0022 - 2s/epoch - 47ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00157 to 0.00148, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2010_best.hdf5
40/40 - 2s 

In [47]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2010
{'units': 400, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0021971656107059785}
2616/2616 [==============================] - 33s 12ms/step - loss: 9.2170e-04 - R2: 0.8722 - mae: 0.0224


[0.0009216971811838448, 0.8721793293952942, 0.022364119067788124]

### 2011

In [48]:
from sklearn.preprocessing import StandardScaler
year = 2011
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (657288, 18, 1)
X_validate shape: (197187, 18, 1)


In [49]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 19s]
val_R2: 0.7073593735694885

Best val_R2 So Far: 0.8749920725822449
Total elapsed time: 01h 32m 24s
INFO:tensorflow:Oracle triggered exit


In [50]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00208, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2011_best.hdf5
41/41 - 2s - loss: 0.0062 - R2: 0.1636 - mae: 0.0546 - val_loss: 0.0021 - val_R2: 0.7177 - val_mae: 0.0355 - lr: 0.0014 - 2s/epoch - 54ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00208 to 0.00163, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2011_best.hdf5
41/41 - 1s - loss: 0.0018 - R2: 0.7502 - mae: 0.0328 - val_loss: 0.0016 - val_R2: 0.7794 - val_mae: 0.0309 - lr: 0.0014 - 1s/epoch - 35ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00163 to 0.00150, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2011_best.hdf5
41/41 - 2s - loss: 0.0015 - R2: 0.7875 - mae: 0.0300 - val_loss: 0.0015 - val_R2: 0.7965 - val_mae: 0.0296 - lr: 0.0014 - 2s/epoch - 40ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00150 to 0.00138, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2011_best.hdf5
41/41 - 2s 

In [51]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2011
{'units': 200, 'activate': 'relu', 'num_layers': 6, 'dropout': False, 'activate_dense_last': 'linear', 'lr': 0.0014421563688159582}
2641/2641 [==============================] - 44s 17ms/step - loss: 9.3019e-04 - R2: 0.8614 - mae: 0.0226


[0.0009301939862780273, 0.8613736033439636, 0.022601677104830742]

### 2012 

In [52]:
from sklearn.preprocessing import StandardScaler
year = 2012
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (660527, 18, 1)
X_validate shape: (198158, 18, 1)


In [53]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 10s]
val_R2: 0.7336162328720093

Best val_R2 So Far: 0.8923549652099609
Total elapsed time: 01h 36m 56s
INFO:tensorflow:Oracle triggered exit


In [54]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00211, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2012_best.hdf5
41/41 - 3s - loss: 0.0061 - R2: 0.2528 - mae: 0.0541 - val_loss: 0.0021 - val_R2: 0.7397 - val_mae: 0.0357 - lr: 0.0015 - 3s/epoch - 71ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00211 to 0.00170, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2012_best.hdf5
41/41 - 2s - loss: 0.0019 - R2: 0.7719 - mae: 0.0331 - val_loss: 0.0017 - val_R2: 0.7897 - val_mae: 0.0313 - lr: 0.0015 - 2s/epoch - 54ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00170 to 0.00153, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2012_best.hdf5
41/41 - 2s - loss: 0.0016 - R2: 0.8036 - mae: 0.0303 - val_loss: 0.0015 - val_R2: 0.8105 - val_mae: 0.0297 - lr: 0.0015 - 2s/epoch - 47ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00153 to 0.00144, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2012_best.hdf5
41/41 - 2s 

In [55]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2012
{'units': 500, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0014808314612806217}
2654/2654 [==============================] - 38s 14ms/step - loss: 8.7431e-04 - R2: 0.8865 - mae: 0.0215


[0.0008743098587729037, 0.8864590525627136, 0.021536942571401596]

### 2013

In [56]:
from sklearn.preprocessing import StandardScaler
year = 2013
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (647136, 18, 1)
X_validate shape: (194140, 18, 1)


In [57]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 03m 20s]
val_R2: 0.8498453497886658

Best val_R2 So Far: 0.8801128268241882
Total elapsed time: 01h 29m 39s
INFO:tensorflow:Oracle triggered exit


In [58]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00230, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2013_best.hdf5
40/40 - 2s - loss: 0.0082 - R2: -4.5538e-02 - mae: 0.0641 - val_loss: 0.0023 - val_R2: 0.7038 - val_mae: 0.0373 - lr: 0.0016 - 2s/epoch - 52ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00230 to 0.00186, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2013_best.hdf5
40/40 - 2s - loss: 0.0020 - R2: 0.7358 - mae: 0.0352 - val_loss: 0.0019 - val_R2: 0.7603 - val_mae: 0.0335 - lr: 0.0016 - 2s/epoch - 41ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00186 to 0.00164, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2013_best.hdf5
40/40 - 2s - loss: 0.0017 - R2: 0.7751 - mae: 0.0322 - val_loss: 0.0016 - val_R2: 0.7885 - val_mae: 0.0312 - lr: 0.0016 - 2s/epoch - 40ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00164 to 0.00151, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2013_best.hdf5
40/40 

In [59]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2013
{'units': 200, 'activate': 'relu', 'num_layers': 6, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0016274878533905327}
2601/2601 [==============================] - 36s 14ms/step - loss: 9.0972e-04 - R2: 0.8741 - mae: 0.0223


[0.0009097231668420136, 0.8740836977958679, 0.022295141592621803]

### 2014

In [60]:
from sklearn.preprocessing import StandardScaler
year = 2014
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (603515, 18, 1)
X_validate shape: (181055, 18, 1)


In [61]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 11s]
val_R2: 0.805347740650177

Best val_R2 So Far: 0.8980550169944763
Total elapsed time: 01h 33m 13s
INFO:tensorflow:Oracle triggered exit


In [62]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00219, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2014_best.hdf5
37/37 - 3s - loss: 0.0061 - R2: 0.2655 - mae: 0.0548 - val_loss: 0.0022 - val_R2: 0.7358 - val_mae: 0.0364 - lr: 3.3965e-04 - 3s/epoch - 74ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00219 to 0.00166, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2014_best.hdf5
37/37 - 2s - loss: 0.0018 - R2: 0.7775 - mae: 0.0331 - val_loss: 0.0017 - val_R2: 0.8009 - val_mae: 0.0311 - lr: 3.3965e-04 - 2s/epoch - 57ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00166 to 0.00148, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2014_best.hdf5
37/37 - 2s - loss: 0.0015 - R2: 0.8140 - mae: 0.0300 - val_loss: 0.0015 - val_R2: 0.8222 - val_mae: 0.0291 - lr: 3.3965e-04 - 2s/epoch - 53ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00148 to 0.00137, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2014_best.hdf5

In [63]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2014
{'units': 500, 'activate': 'relu', 'num_layers': 6, 'dropout': False, 'activate_dense_last': 'linear', 'lr': 0.0003396472862594332}
2425/2425 [==============================] - 32s 13ms/step - loss: 8.6338e-04 - R2: 0.8877 - mae: 0.0217


[0.0008633837569504976, 0.8876842260360718, 0.02166232466697693]

### 2015

In [64]:
from sklearn.preprocessing import StandardScaler
year = 2015
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (606046, 18, 1)
X_validate shape: (181813, 18, 1)


In [65]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 01m 56s]
val_R2: 0.8459322452545166

Best val_R2 So Far: 0.8889215588569641
Total elapsed time: 01h 34m 44s
INFO:tensorflow:Oracle triggered exit


In [66]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00228, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2015_best.hdf5
37/37 - 3s - loss: 0.0066 - R2: 0.1843 - mae: 0.0557 - val_loss: 0.0023 - val_R2: 0.7170 - val_mae: 0.0375 - lr: 0.0016 - 3s/epoch - 73ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00228 to 0.00182, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2015_best.hdf5
37/37 - 2s - loss: 0.0020 - R2: 0.7523 - mae: 0.0347 - val_loss: 0.0018 - val_R2: 0.7734 - val_mae: 0.0329 - lr: 0.0016 - 2s/epoch - 41ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00182 to 0.00163, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2015_best.hdf5
37/37 - 1s - loss: 0.0017 - R2: 0.7863 - mae: 0.0320 - val_loss: 0.0016 - val_R2: 0.7976 - val_mae: 0.0309 - lr: 0.0016 - 1s/epoch - 34ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00163 to 0.00150, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2015_best.hdf5
37/37 - 2s 

In [67]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2015
{'units': 500, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.001614959126027819}
2436/2436 [==============================] - 31s 13ms/step - loss: 8.8898e-04 - R2: -inf - mae: 0.0219


[0.0008889774326235056, -inf, 0.021939480677247047]

### 2016

In [68]:
from sklearn.preprocessing import StandardScaler
year = 2016
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (636232, 18, 1)
X_validate shape: (190870, 18, 1)


In [69]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 01m 44s]
val_R2: 0.7596077919006348

Best val_R2 So Far: 0.8919544219970703
Total elapsed time: 01h 29m 43s
INFO:tensorflow:Oracle triggered exit


In [70]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00244, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2016_best.hdf5
39/39 - 2s - loss: 0.0104 - R2: -2.0837e-01 - mae: 0.0709 - val_loss: 0.0024 - val_R2: 0.7159 - val_mae: 0.0388 - lr: 6.7664e-04 - 2s/epoch - 54ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00244 to 0.00195, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2016_best.hdf5
39/39 - 1s - loss: 0.0022 - R2: 0.7495 - mae: 0.0362 - val_loss: 0.0019 - val_R2: 0.7727 - val_mae: 0.0342 - lr: 6.7664e-04 - 1s/epoch - 27ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00195 to 0.00175, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2016_best.hdf5
39/39 - 1s - loss: 0.0018 - R2: 0.7869 - mae: 0.0331 - val_loss: 0.0017 - val_R2: 0.7964 - val_mae: 0.0322 - lr: 6.7664e-04 - 1s/epoch - 30ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00175 to 0.00160, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2016_best

In [71]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2016
{'units': 300, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0006766435736156065}
2557/2557 [==============================] - 39s 15ms/step - loss: 9.3922e-04 - R2: 0.8825 - mae: 0.0228


[0.0009392154752276838, 0.8824617862701416, 0.022838983684778214]

### 2017

In [72]:
from sklearn.preprocessing import StandardScaler
year = 2017
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (637490, 18, 1)
X_validate shape: (191247, 18, 1)


In [73]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 16s]
val_R2: 0.7474083304405212

Best val_R2 So Far: 0.8976514935493469
Total elapsed time: 01h 38m 35s
INFO:tensorflow:Oracle triggered exit


In [74]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00213, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2017_best.hdf5
39/39 - 3s - loss: 0.0076 - R2: 0.1197 - mae: 0.0596 - val_loss: 0.0021 - val_R2: 0.7534 - val_mae: 0.0356 - lr: 9.9261e-04 - 3s/epoch - 70ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00213 to 0.00166, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2017_best.hdf5
39/39 - 2s - loss: 0.0019 - R2: 0.7826 - mae: 0.0332 - val_loss: 0.0017 - val_R2: 0.8077 - val_mae: 0.0311 - lr: 9.9261e-04 - 2s/epoch - 51ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00166 to 0.00147, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2017_best.hdf5
39/39 - 2s - loss: 0.0016 - R2: 0.8201 - mae: 0.0300 - val_loss: 0.0015 - val_R2: 0.8294 - val_mae: 0.0291 - lr: 9.9261e-04 - 2s/epoch - 53ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00147 to 0.00140, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2017_best.hdf5

In [75]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2017
{'units': 400, 'activate': 'relu', 'num_layers': 6, 'dropout': False, 'activate_dense_last': 'linear', 'lr': 0.000992610117785807}
2562/2562 [==============================] - 46s 18ms/step - loss: 8.3292e-04 - R2: 0.8963 - mae: 0.0211


[0.0008329219417646527, 0.896333634853363, 0.021136390045285225]

### 2018

In [76]:
from sklearn.preprocessing import StandardScaler
year = 2018
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (614272, 18, 1)
X_validate shape: (184282, 18, 1)


In [77]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 42s]
val_R2: 0.8364231586456299

Best val_R2 So Far: 0.9094753265380859
Total elapsed time: 01h 35m 16s
INFO:tensorflow:Oracle triggered exit


In [78]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00210, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2018_best.hdf5
38/38 - 2s - loss: 0.0076 - R2: 0.1403 - mae: 0.0600 - val_loss: 0.0021 - val_R2: 0.7622 - val_mae: 0.0357 - lr: 0.0018 - 2s/epoch - 65ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00210 to 0.00173, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2018_best.hdf5
38/38 - 2s - loss: 0.0019 - R2: 0.7861 - mae: 0.0335 - val_loss: 0.0017 - val_R2: 0.8040 - val_mae: 0.0318 - lr: 0.0018 - 2s/epoch - 50ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00173 to 0.00153, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2018_best.hdf5
38/38 - 1s - loss: 0.0016 - R2: 0.8168 - mae: 0.0308 - val_loss: 0.0015 - val_R2: 0.8259 - val_mae: 0.0298 - lr: 0.0018 - 1s/epoch - 38ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00153 to 0.00141, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2018_best.hdf5
38/38 - 2s 

In [79]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2018
{'units': 400, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.001800627430497124}
2469/2469 [==============================] - 40s 16ms/step - loss: 8.1872e-04 - R2: 0.8998 - mae: 0.0211


[0.000818721717223525, 0.8997644782066345, 0.021060142666101456]

### 2019

In [80]:
from sklearn.preprocessing import StandardScaler
year = 2019
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (625580, 18, 1)
X_validate shape: (187674, 18, 1)


In [81]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 02m 09s]
val_R2: 0.8848790526390076

Best val_R2 So Far: 0.9143610000610352
Total elapsed time: 01h 29m 51s
INFO:tensorflow:Oracle triggered exit


In [82]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00190, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2019_best.hdf5
39/39 - 2s - loss: 0.0057 - R2: 0.3298 - mae: 0.0510 - val_loss: 0.0019 - val_R2: 0.7725 - val_mae: 0.0338 - lr: 0.0024 - 2s/epoch - 50ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00190 to 0.00160, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2019_best.hdf5
39/39 - 1s - loss: 0.0017 - R2: 0.7926 - mae: 0.0321 - val_loss: 0.0016 - val_R2: 0.8083 - val_mae: 0.0306 - lr: 0.0024 - 1s/epoch - 33ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00160 to 0.00142, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2019_best.hdf5
39/39 - 2s - loss: 0.0015 - R2: 0.8199 - mae: 0.0296 - val_loss: 0.0014 - val_R2: 0.8300 - val_mae: 0.0285 - lr: 0.0024 - 2s/epoch - 41ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00142 to 0.00134, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2019_best.hdf5
39/39 - 1s 

In [83]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2019
{'units': 500, 'activate': 'relu', 'num_layers': 5, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0023648890851369585}
2514/2514 [==============================] - 37s 15ms/step - loss: 7.0800e-04 - R2: 0.9087 - mae: 0.0196


[0.0007079950883053243, 0.9086650013923645, 0.019627513363957405]

### 2020

In [84]:
from sklearn.preprocessing import StandardScaler
year = 2020
data_train = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','train_data_'+str(year)+'.csv'))
data_validate = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','validate_data_'+str(year)+'.csv'))
data_test = pd.read_csv(os.path.join('D:/SGYL/SM_Downscaling_data/Train/split/','test_data_'+str(year)+'.csv'))

X_train = data_train.drop(['SM','Aspect'],axis = 1)
y_train = data_train['SM'].copy()   
X_validate = data_validate.drop(['SM','Aspect'],axis = 1)
y_validate = data_validate['SM'].copy()
X_test = data_test.drop(['SM','Aspect'],axis = 1)
y_test = data_test['SM'].copy()

standarder = StandardScaler()
X_train = standarder.fit_transform(X_train)
X_validate = standarder.transform(X_validate)
X_test = standarder.transform(X_test)

X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_validate = X_validate.reshape(X_validate.shape[0],X_validate.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

print('X_train shape:',X_train.shape)
print('X_validate shape:',X_validate.shape)

X_train shape: (709183, 18, 1)
X_validate shape: (212755, 18, 1)


In [85]:
def scheduler(epoch,lr):
    # 每隔30个epoch，学习率减小为原来的1/10
    if epoch % 30 == 0 and epoch != 0:
        print("lr changed to {}".format(lr * 0.1))
        return lr * 0.1
    else :
        return lr

callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=20,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
hypermodel = MyHyperModel()
tuner = keras_tuner.BayesianOptimization(
            hypermodel,
            objective=keras_tuner.Objective('val_R2', direction="max"),
            num_initial_points=50,
            max_trials=40,
            overwrite = True,
            directory='D:/SGYL/SM_results_data/Bayesian_Opt/ANN/',
            project_name=('ANN_'+str(year)))
tuner.search(X_train,y_train,epochs=100,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = [reduce_lr,callback_early_stopping],verbose = 2)

Trial 40 Complete [00h 00m 49s]
val_R2: -6.870645523071289

Best val_R2 So Far: 0.9104102849960327
Total elapsed time: 01h 51m 27s
INFO:tensorflow:Oracle triggered exit


In [86]:
filepath="ANN_"+str(year)+"_best.hdf5"
callback_checkpoints = keras.callbacks.ModelCheckpoint(os.path.join('D:/SGYL/SM_results_data/check_points/ANN/',filepath),monitor='val_loss',save_best_only=True,verbose=1)
callback_early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',patience=50,verbose=1)
reduce_lr = keras.callbacks.LearningRateScheduler(scheduler)
callbacks = [reduce_lr,callback_early_stopping,callback_checkpoints]

if training_mode == 'model':
    model_best = tuner.get_best_models()[0]
else :
    model_best = tuner.hypermodel.build(tuner.get_best_hyperparameters()[0])
model_best.fit(X_train,y_train,epochs=300,batch_size = batch_size,validation_data=(X_validate,y_validate),shuffle = True,callbacks = callbacks,verbose = 2)

Epoch 1/300

Epoch 1: val_loss improved from inf to 0.00199, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2020_best.hdf5
44/44 - 3s - loss: 0.0061 - R2: 0.2285 - mae: 0.0541 - val_loss: 0.0020 - val_R2: 0.7446 - val_mae: 0.0344 - lr: 0.0012 - 3s/epoch - 58ms/step
Epoch 2/300

Epoch 2: val_loss improved from 0.00199 to 0.00156, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2020_best.hdf5
44/44 - 2s - loss: 0.0017 - R2: 0.7774 - mae: 0.0320 - val_loss: 0.0016 - val_R2: 0.8003 - val_mae: 0.0303 - lr: 0.0012 - 2s/epoch - 45ms/step
Epoch 3/300

Epoch 3: val_loss improved from 0.00156 to 0.00136, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2020_best.hdf5
44/44 - 2s - loss: 0.0015 - R2: 0.8131 - mae: 0.0292 - val_loss: 0.0014 - val_R2: 0.8257 - val_mae: 0.0281 - lr: 0.0012 - 2s/epoch - 44ms/step
Epoch 4/300

Epoch 4: val_loss improved from 0.00136 to 0.00126, saving model to D:/SGYL/SM_results_data/check_points/ANN\ANN_2020_best.hdf5
44/44 - 2s 

In [87]:
#save model
save_path = os.path.join('D:/SGYL/SM_results_data/model/ANN/','ANN_'+str(year)+'.hdf5')
model_best.save(save_path)
if (os.path.exists(save_path)):
    print('save model for year:',year)

#save model config
import json
bestConfig=tuner.get_best_hyperparameters()[0].get_config()
f = open(os.path.join('D:/SGYL/SM_results_data/Bayesian_Opt/ANN/Best_Config/','ANN_'+str(year)+'.json'), 'w')
json.dump(bestConfig,f)
f.close()
print(bestConfig['values'])

model_best.evaluate(X_test,y_test)

save model for year: 2020
{'units': 400, 'activate': 'relu', 'num_layers': 6, 'dropout': False, 'activate_dense_last': 'sigmoid', 'lr': 0.0011890424459478066}
2850/2850 [==============================] - 48s 17ms/step - loss: 7.0094e-04 - R2: 0.9030 - mae: 0.0195


[0.0007009362452663481, 0.9030131101608276, 0.019522717222571373]

### aaa